In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow import keras

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
def dfd_image(input_shape, units):
    inception_resnet_v2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False,
                                                                                   weights='imagenet', 
                                                                                   input_tensor=None, 
                                                                                   input_shape=input_shape, 
                                                                                   pooling=None, 
                                                                                   classes=1000)
    
    for layer in inception_resnet_v2.layers[:-80]:
        layer.trainable = False
    
    model = tf.keras.Sequential()

    model.add(inception_resnet_v2)
    
    model.add(GlobalAveragePooling2D())
    
    model.add(Flatten())
    
    model.add(Dense(units=128, activation='relu'))
    
    model.add(Dense(units, kernel_regularizer=l2(0.01), activation='linear'))
    
    return model

In [4]:
input_shape = (256, 256, 3)

In [5]:
model = dfd_image(input_shape, units=1)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               196736    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 54,533,601
Trainable params: 13,166,529
Non-trainable params: 41,367,072
_________________________________________________________________


In [7]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision * recall)/(precision + recall + K.epsilon()))

In [8]:
metrics = ['accuracy', f1_score_m, precision_m, recall_m]

model.compile(optimizer = 'adam',
              loss = 'hinge',
              metrics = metrics)

In [9]:
path = 'Weights_Inception_Resnet_SVM/model_weights'
model.load_weights(path)

In [10]:
import os

In [11]:
for files in os.walk('test'):
    print(files)

('test', ['Fake', 'Real'], [])
('test\\Fake', [], ['0AEIDNSBKD.jpg', '0IGA28OUAK.jpg', '0QOGCHTX2O.jpg', '0QW46QAHNF.jpg', '1DKCGEETES.jpg', 'image.10jfif.jfif', 'image1.jfif', 'image10.jfif', 'image11.jfif', 'image12.jfif', 'image13.jfif', 'image14.jfif', 'image15.jfif', 'image16.jfif', 'image17.jfif', 'image18.jfif', 'image19.jfif', 'image2.jfif', 'image20.jfif', 'image3.jfif', 'image4.jfif', 'image5.jfif', 'image6.jfif', 'image7.jfif', 'image8.jfif', 'image9.jfif'])
('test\\Real', [], ['00127.jpg', '01041.jpg', '01044.jpg', '02405.jpg', '03503.jpg', 'download.4png.png', 'download1.png', 'download2.png', 'download3.png', 'download4.png', 'download5.png', 'download6.png', 'download7.png', 'WIN_20220117_14_20_53_Pro.jpg'])


In [12]:
def predict(path):
    img_path = path
    img = image.load_img(img_path, target_size=(256, 256))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.image.resize(x, (256,256))
    x /= 255

    res = model.predict(x)
    
    if(res[0][0]<0):
        print("Fake")
    else:
        print("Real")

In [13]:
path = 'test/Fake/image20.jfif'
predict(path)

Fake
